In [102]:
if (!requireNamespace("BiocManager", quietly = TRUE)) {install.packages("BiocManager", repos = "http://cran.us.r-project.org")}

if ("DESeq2" %in% rownames(installed.packages()) == FALSE) {
  BiocManager::install("DESeq2", dependencies=c("Depends", "Imports", "LinkingTo"))
} else {
  print("DESeq2 package already installed")
}

if ("apeglm" %in% rownames(installed.packages()) == FALSE) {
  BiocManager::install("apeglm", dependencies=c("Depends", "Imports", "LinkingTo"))
} else {
  print("apeglm package already installed")
}

if ("biomaRt" %in% rownames(installed.packages()) == FALSE) {
  BiocManager::install("biomaRt", dependencies=c("Depends", "Imports", "LinkingTo"))
} else {
  print("biomaRt package already installed")
}

library(DESeq2)
library(apeglm)
library(biomaRt)

[1] "DESeq2 package already installed"
[1] "apeglm package already installed"
[1] "biomaRt package already installed"


In [7]:
getwd()

[1] "C:/Users/jorda/Desktop/projects/winter_analysis_2022"

In [73]:
DATAFRAME <- "data\\winter_rnaseq_xenografts_count_table.tsv"
INFO <- "data\\winter_2022_xenografts_metadata.txt"

In [74]:
# Import counts_data
count_table <- read.table(DATAFRAME, sep = '\t', header = TRUE, row.names = 1, check.names=F)
colnames(count_table) <- sub("_.*", "", colnames(count_table))

# Create conditions dataframe
sample_table <- read.table(text = readLines(INFO, warn = FALSE), header = TRUE, sep = '\t')

In [75]:
conditions <- list(
    "NT_Sal",
    "NT_BTZ",
    "ATAD1_Sal",
    "ATAD1_BTZ")

In [103]:
mart <- useMart(biomart = "ensembl", dataset = "hsapiens_gene_ensembl")

In [104]:
IDs <- getBM(
    attributes = c("ensembl_gene_id","hgnc_symbol"),
    filters = "ensembl_gene_id", 
    values = rownames(count_table),
    mart = mart)


Batch submitting query [=>-----------------------------]   8% eta:  4m

Batch submitting query [====>--------------------------]  15% eta:  3m

Batch submitting query [======>------------------------]  23% eta:  3m

Batch submitting query [=========>---------------------]  31% eta:  2m

Batch submitting query [===========>-------------------]  38% eta:  1m

Batch submitting query [=============>-----------------]  46% eta:  1m

Batch submitting query [================>--------------]  54% eta: 45s

Batch submitting query [==================>------------]  62% eta: 34s

Batch submitting query [====================>----------]  69% eta: 27s

Batch submitting query [=======================>-------]  77% eta: 20s

Batch submitting query [=========================>-----]  85% eta: 13s

Batch submitting query [============================>--]  92% eta:  7s
                                                                      



In [124]:
for (c1 in conditions) {
    for (c2 in conditions) {
        if (c1 != c2) {
            print(paste("Analyzing", c2, "vs", c1, sep=" "))
            OUTPUT <- paste("results\\", "deseq_results_", c2, "_vs_", c1, ".tsv", sep="")

            select_samples <- sample_table[sample_table$Condition == c1 | sample_table$Condition == c2, ]
            select_counts <- count_table[select_samples$Sample.names]

            # Run DESeq2 analysis on data
            dds <- DESeqDataSetFromMatrix(
                countData = select_counts,
                colData = select_samples,
                design = ~ Condition)
            dds <- DESeq(dds)
            res <- results(dds, contrast=c("Condition", c2, c1))
            
            # Write output to new file
            output_table <- as.data.frame(res)
            export_table <- merge(
                x = IDs, 
                y = output_table,
                by.x = "ensembl_gene_id", 
                by.y = 0
            )
            resOrdered <- export_table[order(export_table$padj),]
            write.table(as.data.frame(resOrdered), file = OUTPUT, sep = '\t', row.names = FALSE)
        }
    }
}

[1] "Analyzing NT_BTZ vs NT_Sal"


Warning message in DESeqDataSet(se, design = design, ignoreRank):
"some variables in design formula are characters, converting to factors"
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Analyzing ATAD1_Sal vs NT_Sal"


Warning message in DESeqDataSet(se, design = design, ignoreRank):
"some variables in design formula are characters, converting to factors"
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Analyzing ATAD1_BTZ vs NT_Sal"


...

Warning message in DESeqDataSet(se, design = design, ignoreRank):
"some variables in design formula are characters, converting to factors"
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Analyzing NT_BTZ vs ATAD1_BTZ"


Warning message in DESeqDataSet(se, design = design, ignoreRank):
"some variables in design formula are characters, converting to factors"
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Analyzing ATAD1_Sal vs ATAD1_BTZ"


Warning message in DESeqDataSet(se, design = design, ignoreRank):
"some variables in design formula are characters, converting to factors"
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

